# Regression Week 2: Multiple Regression (gradient descent)

In the first notebook we explored multiple regression using graphlab create. Now we will use graphlab along with numpy to solve for the regression weights with gradient descent.

In this notebook we will cover estimating multiple regression weights via gradient descent. You will:
* Add a constant column of 1's to a graphlab SFrame to account for the intercept
* Convert an SFrame into a Numpy array
* Write a predict_output() function using Numpy
* Write a numpy function to compute the derivative of the regression weights with respect to a single feature
* Write gradient descent function to compute the regression weights given an initial weight vector, step size and tolerance.
* Use the gradient descent function to estimate regression weights for multiple features

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [ ]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [1]:
#sales = graphlab.SFrame('kc_house_data.gl/')
import pandas
import sklearn
#print sklearn.__version__
#pip list | grep sklearn
#pip install --upgrade sklearn
from sklearn.model_selection import train_test_split

sales = pandas.read_csv('kc_house_data.csv')


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the other Week 2 notebook. For this notebook, however, we will work with the existing features.

# Convert to Numpy Array

Although SFrames offer a number of benefits to users (especially when using Big Data and built-in graphlab functions) in order to understand the details of the implementation of algorithms it's important to work with a library that allows for direct (and optimized) matrix operations. Numpy is a Python solution to work with matrices (or any multi-dimensional "array").

Recall that the predicted value given the weights and the features is just the dot product between the feature and weight vector. Similarly, if we put all of the features row-by-row in a matrix then the predicted value for *all* the observations can be computed by right multiplying the "feature matrix" by the "weight vector". 

First we need to take the SFrame of our data and convert it into a 2D numpy array (also called a matrix). To do this we use graphlab's built in .to_dataframe() which converts the SFrame into a Pandas (another python library) dataframe. We can then use Panda's .as_matrix() to convert the dataframe into a numpy matrix.

In [2]:
import numpy as np # note this allows us to refer to numpy as np instead 

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:
* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')
* A numpy array containing the values of the output

With this in mind, complete the following function (where there's an empty line you should write a line of code that does what the comment above indicates)

**Please note you will need GraphLab Create version at least 1.7.1 in order for .to_numpy() to work!**

In [3]:
sales

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [3]:
from pandas import Series
def get_numpy_data(data_sframe, features):
    #data_sframe['constant'] = Series(np.random.randn(sLength), index=data_sframe.index)1 # this is how you add a constant column to an SFrame
    data_sframe = data_sframe.assign(constant=Series(np.ones(len(data_sframe.index))).values)
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    #feature_matrix = features_sframe.to_numpy()
    feature_matrix = features_sframe.values
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    #output_array = data_sframe['price'].values
    #output_array = output.values
    # the following will convert the SArray into a numpy array by first converting it to a list
    #output_array = output_sarray.to_numpy()
    #return(feature_matrix, output_array)
    return feature_matrix

For testing let's use the 'sqft_living' feature and a constant as our features and price as our output:

In [4]:
data_sframe = sales
temp=np.ones(len(data_sframe.index))
#temp
temp2 = Series(temp)
data_sframe = data_sframe.assign(constant=temp2.values)

In [5]:


example_features = get_numpy_data(sales, ['sqft_living']) # the [] around 'sqft_living' makes it a list
example_output = sales['price'].values
print example_features[0,:] # this accesses the first row of the data the ':' indicates 'all columns'
print example_output[0] # and the corresponding output

[  1.00000000e+00   1.18000000e+03]
221900.0


# Predicting output given regression weights

Suppose we had the weights [1.0, 1.0] and the features [1.0, 1180.0] and we wanted to compute the predicted output 1.0\*1.0 + 1.0\*1180.0 = 1181.0 this is the dot product between these two arrays. If they're numpy arrayws we can use np.dot() to compute this:

In [6]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print predicted_value

1181.0


np.dot() also works when dealing with a matrix and a vector. Recall that the predictions from all the observations is just the RIGHT (as in weights on the right) dot product between the features *matrix* and the weights *vector*. With this in mind finish the following predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [7]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

If you want to test your code run the following cell:

In [8]:
test_predictions = predict_output(example_features, my_weights)
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1181.0
2571.0


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [9]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative = 2*np.dot(errors,feature)
    return(derivative)

To test your feature derivartive run the following:

In [10]:
example_features = get_numpy_data(sales, ['sqft_living']) 
example_output = sales['price'].values
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"



In [11]:
np.dot(errors, feature)

-11672925008.0

In [12]:
feature

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [13]:
example_features = get_numpy_data(sales, ['sqft_living']) 
example_output = sales['price'].values
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print derivative
print -np.sum(example_output)*2 # should be the same as derivative

-23345850016.0
-23345850016.0


# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [14]:
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [15]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        errors = predictions - output
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_squares += derivative**2
            # subtract the step size times the derivative from the current weight
            weights[i] -= step_size*derivative
        # compute the square-root of the gradient sum of squares to get the gradient magnitude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        print gradient_magnitude
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

A few things to note before we run the gradient descent. Since the gradient is a sum over all the data points and involves a product of an error and a feature the gradient itself will be very large since the features are large (squarefeet) and the output is large (prices). So while you might expect "tolerance" to be small, small is only relative to the size of the features. 

For similar reasons the step size will be much smaller than you might expect but this is because the gradient has such large values.

# Running the Gradient Descent as Simple Regression

First let's split the data into training and test data.

In [16]:
#train_data,test_data = sales.random_split(.8,seed=0)
example_features = ['sqft_living']
target = ['price']

X_train, X_test, y_train, y_test = train_test_split(sales, sales[target], test_size=0.2, random_state=0)

Although the gradient descent is designed for multiple regression since the constant is now a feature we can use the gradient descent function to estimat the parameters in the simple regression on squarefeet. The folowing cell sets up the feature_matrix, output, initial weights and step size for the first model:

In [21]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
train_data = X_train.merge(y_train)
test_data = X_test.merge(y_test)
#train_data.merge(y_train)



Next run your gradient descent with the above parameters.

In [18]:
#(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
simple_feature_matrix = get_numpy_data(train_data, simple_features) 
output = train_data['price'].values
initial_weights = np.array([-47000., 1.])
step_size = 7e-14
tolerance = 3.7e10

In [19]:
weight = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

1.30760812158e+15
8.22083376989e+14
5.16837627567e+14
3.24931924634e+14
2.04282255471e+14
1.28430718443e+14
8.07434303454e+13
5.07627945128e+13
3.19141993504e+13
2.00642371147e+13
1.26142671513e+13
7.93054629392e+12
4.98595671725e+12
3.13476385558e+12
1.97100986116e+12
1.23948894287e+12
7.79781839207e+11
4.91076214927e+11
3.10057328905e+11
1.97016905774e+11
1.27120943858e+11
84881586372.3
60542887150.9
47607368947.5
41394552399.8
38664610411.3
37530874688.7
37073201723.9
36890738693.5


How do your weights compare to those achieved in week 1 (don't expect them to be exactly the same)? 

**Quiz Question: What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

In [20]:
weight

array([-46999.81261389,    247.51375885])

Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [23]:
#(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)
test_simple_feature_matrix = get_numpy_data(test_data, simple_features) 
output = test_data['price'].values


Now compute your predictions using test_simple_feature_matrix and your weights from above.

In [25]:
test_predictions = predict_output(test_simple_feature_matrix, weight)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?**

In [27]:
errors = test_predictions - output
errors[0]/output[0]
sqrt(sum(errors**2)/len(errors))

0.033484385654166213

In [29]:
errors

array([   9944.86253929,    9944.86253929,    9944.86253929, ...,
       -126182.29777241,  -70632.47646494, -217027.24810255])

Now that you have the predictions on test data, compute the RSS on the test data set. Save this value for comparison later. Recall that RSS is the sum of the squared errors (difference between prediction and output).

In [30]:
sqrt(sum(errors**2)/len(errors))

184808.8987844524

# Running a multiple regression

Now we will use more than one actual feature. Use the following code to produce the weights for a second model with the following parameters:

In [44]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
feature_matrix = get_numpy_data(train_data, model_features) 
output = train_data['price'].values
#(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 7e-14
tolerance = 2e9
#step_size = 7e-14
#tolerance = 3.7e10

Use the above parameters to estimate the model weights. Record these values for your quiz.

In [45]:
multi_weight = regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)

1.98391694923e+15
5.98685162933e+14
1.8075516984e+14
5.48651214075e+13
1.75627844781e+13
7.85106689455e+12
6.19510405347e+12
5.95763585485e+12
5.872026878e+12
5.80138884854e+12
5.7328719879e+12
5.6652815604e+12
5.59849884736e+12
5.53250439474e+12
5.46728799542e+12
5.40284039346e+12
5.33915251864e+12
5.27621541469e+12
5.21402023156e+12
5.15255822362e+12
5.09182074831e+12
5.03179926496e+12
4.97248533358e+12
4.9138706137e+12
4.85594686312e+12
4.79870593684e+12
4.74213978587e+12
4.68624045608e+12
4.63100008715e+12
4.57641091137e+12
4.52246525264e+12
4.46915552532e+12
4.41647423322e+12
4.36441396848e+12
4.31296741061e+12
4.26212732539e+12
4.21188656387e+12
4.16223806142e+12
4.11317483663e+12
4.06468999044e+12
4.01677670509e+12
3.96942824321e+12
3.92263794684e+12
3.87639923651e+12
3.83070561032e+12
3.78555064299e+12
3.74092798503e+12
3.69683136176e+12
3.6532545725e+12
3.61019148963e+12
3.5676360578e+12
3.52558229303e+12
3.48402428189e+12
3.44295618064e+12
3.40237221446e+12
3.36226667658e+12


KeyboardInterrupt: 

Use your newly estimated weights and the predict_output function to compute the predictions on the TEST data. Don't forget to create a numpy array for these features from the test set first!

In [40]:
#(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)
test_model_feature_matrix = get_numpy_data(test_data, model_features) 
output = test_data['price'].values

test_predictions = predict_output(test_model_feature_matrix, multi_weight)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

In [41]:
errors = test_predictions - output
errors[0]/output[0]


0.033007427088363822

What is the actual price for the 1st house in the test data set?

In [42]:
output[0]

297000.0

**Quiz Question: Which estimate was closer to the true price for the 1st house on the TEST data set, model 1 or model 2?**

Now use your predictions and the output to compute the RSS for model 2 on TEST data.

In [43]:
sqrt(sum(errors**2)/len(errors))

175986.46791523378

**Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data? **